## UBER DATA END to END DATA ENGINEERING PIPELINE

In [36]:
## Import required libraries
import pandas as pd
import os
import datetime as dt
import numpy as np
import time as tm

#### Load Data From CSV File

In [133]:
## Load Data From CSV File
df = pd.read_csv('data/uber_data.csv')

In [134]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80


#### CREATE DATETIME DIM TABLE

In [135]:
## Convert pickup and dropoff datetime columns to datetime type
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [136]:
## Create datetime dimension table from the main dataframe using pickup and dropoff datetime columns
df_datetime_dim = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].drop_duplicates().reset_index(drop=True)


In [137]:
## Extract various datetime attributes for pickup datetime
df_datetime_dim['pickup_date'] = df_datetime_dim['tpep_pickup_datetime'].dt.date
df_datetime_dim['pickup_time'] = df_datetime_dim['tpep_pickup_datetime'].dt.time
df_datetime_dim['pickup_hour'] = df_datetime_dim['tpep_pickup_datetime'].dt.hour
df_datetime_dim['pickup_day'] = df_datetime_dim['tpep_pickup_datetime'].dt.day_name()
df_datetime_dim['pickup_month'] = df_datetime_dim['tpep_pickup_datetime'].dt.month_name()
df_datetime_dim['pickup_year'] = df_datetime_dim['tpep_pickup_datetime'].dt.year
df_datetime_dim['pickup_weekday'] = df_datetime_dim['tpep_pickup_datetime'].dt.weekday
## Extract various datetime attributes for dropoff datetime
df_datetime_dim['dropoff_date'] = df_datetime_dim['tpep_dropoff_datetime'].dt.date
df_datetime_dim['dropoff_time'] = df_datetime_dim['tpep_dropoff_datetime'].dt.time
df_datetime_dim['dropoff_hour'] = df_datetime_dim['tpep_dropoff_datetime'].dt.hour
df_datetime_dim['dropoff_day'] = df_datetime_dim['tpep_dropoff_datetime'].dt.day_name()
df_datetime_dim['dropoff_month'] = df_datetime_dim['tpep_dropoff_datetime'].dt.month_name()
df_datetime_dim['dropoff_year'] = df_datetime_dim['tpep_dropoff_datetime'].dt.year
df_datetime_dim['dropoff_weekday'] = df_datetime_dim['tpep_dropoff_datetime'].dt.weekday

## Trip duration in minutes (rounded and as integer)
df_datetime_dim['trip_duration_minutes'] = ((df_datetime_dim['tpep_dropoff_datetime'] - df_datetime_dim['tpep_pickup_datetime']).dt.total_seconds() / 60).round().astype(int)

## Create datetime_id as the index
df_datetime_dim['datetime_id'] = df_datetime_dim.index + 1

## Rearranging columns
df_datetime_dim = df_datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'pickup_date', 'pickup_time', 'pickup_hour', 'pickup_day', 'pickup_month', 'pickup_year', 'pickup_weekday',
                                   'dropoff_date', 'dropoff_time', 'dropoff_hour', 'dropoff_day', 'dropoff_month', 'dropoff_year', 'dropoff_weekday',
                                   'trip_duration_minutes']]

#### CREATE PASSENGER COUNT DIM TABLE

In [138]:
## Passenger count dimension
df_passenger_count_dim = df['passenger_count'].drop_duplicates().sort_values().reset_index(drop=True)
## 
df_passenger_count_dim = pd.DataFrame({'passenger_count': df_passenger_count_dim})
## 
df_passenger_count_dim['passenger_count_id'] = df_passenger_count_dim.index + 1
## 
df_passenger_count_dim = df_passenger_count_dim[['passenger_count_id', 'passenger_count']]



#### CREATE TRIP DISTANCE DIM TABLE

In [139]:
# Trip distance dimension
df_trip_distance_dim = df['trip_distance'].drop_duplicates().sort_values().reset_index(drop=True)
## 
df_trip_distance_dim = pd.DataFrame({'trip_distance': df_trip_distance_dim})
## 
df_trip_distance_dim['trip_distance_id'] = df_trip_distance_dim.index + 1
## 
df_trip_distance_dim = df_trip_distance_dim[['trip_distance_id', 'trip_distance']]

#### CREATE RATE CODE DIm TABLE

In [140]:
## rate code mapping 
rate_code_mapping = {
    1: 'Standard rate',
    2: 'JFK',
    3: 'Newark',
    4: 'Nassau or Westchester',
    5: 'Negotiated fare',
    6: 'Group ride'
}

## Rate code dimension table
df_rate_code_dim = df['RatecodeID'].drop_duplicates().sort_values().reset_index(drop=True)
## Create a DataFrame with RatecodeID column
df_rate_code_dim = pd.DataFrame({'RatecodeID': df_rate_code_dim})
## Map RatecodeID to descriptions
df_rate_code_dim['rate_code_description'] = df_rate_code_dim['RatecodeID'].map(rate_code_mapping)
## Create rate_code_id as the index
df_rate_code_dim['rate_code_id'] = df_rate_code_dim.index + 1 

#### CREATE PAYMENT TYPE DIM TABLE

In [141]:
## Payment type mapping
payment_type_mapping = {
    1: 'Credit card',
    2: 'Cash',
    3: 'No charge',
    4: 'Dispute',
    5: 'Unknown',
    6: 'Voided trip'
}
## Payment type dimension table
df_payment_type_dim = df['payment_type'].drop_duplicates().sort_values().reset_index(drop=True)
## Create a DataFrame with payment_type column
df_payment_type_dim = pd.DataFrame({'payment_type': df_payment_type_dim})
## Map payment_type to descriptions
df_payment_type_dim['payment_type_description'] = df_payment_type_dim['payment_type'].map(payment_type_mapping)
## Create payment_type_id as the index
df_payment_type_dim['payment_type_id'] = df_payment_type_dim.index + 1

#### CREATE PICKUP LOCATIOn DIM TABLE

In [142]:
## Pickup location dimension table
df_pickup_location_dim = df[['pickup_longitude', 'pickup_latitude']].drop_duplicates().sort_values(by=['pickup_longitude', 'pickup_latitude']).reset_index(drop=True)
## Create pickup_location_id as the index
df_pickup_location_dim['pickup_location_id'] = df_pickup_location_dim.index + 1
## Rearranging columns
df_pickup_location_dim = df_pickup_location_dim[['pickup_location_id', 'pickup_longitude', 'pickup_latitude']]

#### CREATE DROP LOCATION DIM TABLE

In [143]:
## create drop location dimension table
df_dropoff_location_dim = df[['dropoff_longitude', 'dropoff_latitude']].drop_duplicates().sort_values(by=['dropoff_longitude', 'dropoff_latitude']).reset_index(drop=True)
## Create dropoff_location_id as the index
df_dropoff_location_dim['dropoff_location_id'] = df_dropoff_location_dim.index + 1
## Rearranging columns
df_dropoff_location_dim = df_dropoff_location_dim[['dropoff_location_id', 'dropoff_longitude', 'dropoff_latitude']]

#### CREATE FACT TABLE

In [ ]:
## create fact table 
df_fact_table = df.merge(df_datetime_dim, on= ['tpep_pickup_datetime', 'tpep_dropoff_datetime'], how='left') \
                  .merge(df_passenger_count_dim, on=['passenger_count'], how='left') \
                  .merge(df_trip_distance_dim, on=['trip_distance'], how ='left') \
                  .merge(df_rate_code_dim, on=['RatecodeID'], how='left') \
                  .merge(df_payment_type_dim, on=['payment_type'], how='left') \
                  .merge(df_pickup_location_dim, on=['pickup_longitude', 'pickup_latitude'], how='left') \
                  .merge(df_dropoff_location_dim, on=['dropoff_longitude', 'dropoff_latitude'], how='left') \
                 [['VendorID', 'datetime_id', 'passenger_count_id', 'trip_distance_id', 'rate_code_id', 'payment_type_id',
                    'pickup_location_id', 'dropoff_location_id', 'store_and_fwd_flag', 'fare_amount', 'extra', 
                    'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']]

: 